In [1]:
import cv2
#crop รูปจากวีดีโอและบันทึก

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
def creat_dataset(img,id,img_id):
        cv2.imwrite("test/banana."+str(id)+"."+str(img_id)+".jpg",img)

def draw_boundary(img,classifier,scaleFactor,minNeighbors,color,text):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray,scaleFactor,minNeighbors)
        coords = []
        for (x,y,w,h) in features :
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.putText(img,text,(x,y-4),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,2)
            coords=[x,y,w,h]
        return img,coords

def detect (img,faceCascade,img_id):
        img,coords = draw_boundary(img,faceCascade,1.1,10,(0,0,255),"Face")
        id = 6
        creat_dataset (img,id,img_id)
        return img

#open camera (เปิดใช้งานกล้อง)
img_id = 0
cap = cv2.VideoCapture(0)
while (True):
    ret,frame = cap.read()
    frame = detect(frame,faceCascade,img_id)
    cv2.imshow('frame',frame)
    
    if (cv2.waitKey(1) & 0xFF== ord('q')):
        break
cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np
#เปิดกล้องที่ใช้บันทึก
cap = cv2.VideoCapture(0)

while (cap.isOpened()):
    ref,frame =cap.read()
#ตัดสัญญาณลบกวน เกาส์เซียน    
    roi = frame[:1500,0:2000]#บริเวณที่ต้องการ
#แปลงภาพแบบสีเทา
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
#remove noise
    gray_blur = cv2.GaussianBlur(gray,(15,15),0)
    thresh = cv2.adaptiveThreshold(gray_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,1)
#สร้างเมทริกซ์เติมเต็ม
    kernel = np.ones((3,3),np.uint8)
    closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel,iterations=4)

    result_img =closing.copy()
    contours,hierachy = cv2.findContours(result_img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) #หาขอบ
    for cnt in contours :
        area = cv2.contourArea(cnt)
        if area<5000 or area>35000:
            continue
        ellipse =cv2.fitEllipse(cnt)
        cv2.ellipse(roi,ellipse,(0,255,0),2)


    cv2.imshow("Show",roi)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


[ WARN:0] global /tmp/pip-req-build-afu9cjzs/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index
